In [1]:
import pyspark
import pandas as pd
from pca import pca
from typing import Union, List, Dict
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, BooleanType
from pyspark.sql.functions import pandas_udf, PandasUDFType, lit, col, when, sum as spark_sum, monotonically_increasing_id

import pyspark
import pandas as pd
from pca import pca
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType
from pyspark.sql.functions import pandas_udf, PandasUDFType, lit, col, countDistinct, when, monotonically_increasing_id, \
    sum as spark_sum, collect_set, explode
from typing import List, Dict, Union

In [3]:
import os
import json
from pyspark.sql import SparkSession
import pyspark.pandas as ps

os.environ['PYSPARK_PYTHON'] = '/usr/local/python-3.9.13/bin/python3'
spark = SparkSession.builder \
    .appName("pandas_udf") \
    .config('spark.sql.session.timeZone', 'Asia/Shanghai') \
    .config("spark.scheduler.mode", "FAIR") \
    .config('spark.driver.memory', '8g') \
    .config('spark.driver.cores', '12') \
    .config('spark.executor.memory', '8g') \
    .config('spark.executor.cores', '12') \
    .config('spark.cores.max', '12') \
    .config('spark.driver.host', '192.168.22.28') \
    .master("spark://192.168.12.47:7077,192.168.12.48:7077") \
    .getOrCreate()

[scatterd] >WARNING> 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.


In [123]:
df_pandas = pd.read_csv(
    "D:/Jupyterfiles/晶合MVAFDC_general开发/MVAanlysisDevelop/inline_algorithm/inline_case5_label.csv")
df_pandas = df_pandas[
    df_pandas['OPE_NO'].isin(['1V.EQW10', '1V.PQW10', '1F.FQE10', '1C.CDG10', '1U.EQW10', '1U.PQW10'])]
df_spark = ps.from_pandas(df_pandas).to_spark()
df_spark = df_spark.withColumnRenamed('OPE_NO', 'OPER_NO')
print(f"df_spark shape: ({df_spark.count()}, {len(df_spark.columns)})")
df_spark.show()

C:\Users\yang.wenjun\AppData\Local\Temp\ipykernel_11108\3289620642.py:1: DtypeWarning: Columns (134) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pandas = pd.read_csv(
D:\developer\Anaconda\lib\site-packages\pyspark\pandas\internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
D:\developer\Anaconda\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
D:\developer\Anaconda\lib\site-packages\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


df_spark shape: (2461, 144)
+---------+--------+-------------------+-------------------+-----------+------+--------------+---------+-----------------+---------+---------+----------+---------------+-------------+-------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+--------------+--------+------------+-------+-------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+-------

In [5]:
df_pandas.groupby('OPE_NO')['INLINE_PARAMETER_ID'].nunique()

OPE_NO
1C.CDG10     11
1F.FQE10     15
1U.EQW10     97
1U.PQW10     74
1V.EQW10    117
1V.PQW10    270
Name: INLINE_PARAMETER_ID, dtype: int64

In [6]:
sum(df_pandas.groupby('OPE_NO')['INLINE_PARAMETER_ID'].nunique())

584

In [7]:
set1 = (set(df_pandas[df_pandas['OPE_NO'] == '1U.EQW10']['INLINE_PARAMETER_ID'].values))
print(len(set1))

set2 = (set(df_pandas[df_pandas['OPE_NO'] == '1U.PQW10']['INLINE_PARAMETER_ID'].values))
print(len(set2))

print(len(set1.intersection(set2)))
print(len(set1.difference(set2)))

97
74
74
23


In [37]:
# grouped = df_pandas.groupby('OPE_NO')['INLINE_PARAMETER_ID']
# unique_values_within_groups = grouped.apply(lambda x: x.drop_duplicates()).reset_index(drop=False)
# unique_values_within_groups = unique_values_within_groups[['OPE_NO', 'INLINE_PARAMETER_ID']]
# # unique_values_within_groups

# len(unique_values_within_groups[unique_values_within_groups['OPE_NO'] == '1U.EQW10'])

In [128]:
# grouped = df_spark.groupby('OPER_NO').agg(collect_set('INLINE_PARAMETER_ID').alias('unique_values'))
# exploded = grouped.select(col('OPER_NO'), explode(col('unique_values')).alias('INLINE_PARAMETER_ID'))
# unique_params_within_groups = exploded.dropDuplicates()
# unique_params_within_groups.show()

In [124]:
json_loads_dict = {"requestId": "269",
                   "algorithm": "inline_by_wafer",
                   "requestParam": {"dateRange": {"start": "2021-12-06 19:50:49", "end": "2024-03-06 19:50:49"},
                                    "operNo": [],
                                    "uploadId": "84f6a2b46a5443ec9797347424402058",
                                    "flagMergeAllProdg1": "0",
                                    "flagMergeAllProductId": "0",
                                    "flagMergeAllChamber": "0",
                                    "mergeProdg1": [],
                                    "mergeProductId": [],
                                    "mergeEqp": [],
                                    "mergeChamber": [],

#                                     "mergeOperno": [{"1F.FQE10,1C.CDG10": ["1F.FQE10", "1C.CDG10"]},
#                                                     {"1U.EQW10_1U.PQW10": ["1U.EQW10", "1U.PQW10"]}],

                                    "mergeOperno": None
                                    }
                   }

df_ = pd.DataFrame({"requestId": [json_loads_dict["requestId"]],
                    "requestParam": [json.dumps(json_loads_dict["requestParam"])]})

request_id_ = df_["requestId"].values[0]
request_params = df_["requestParam"].values[0]
parse_dict = json.loads(request_params)

merge_operno = list(parse_dict.get('mergeOperno')) if parse_dict.get('mergeOperno') else None
merge_prodg1 = list(parse_dict.get('mergeProdg1')) if parse_dict.get('mergeProdg1') else None
merge_product = list(parse_dict.get('mergeProductId')) if parse_dict.get('mergeProductId') else None
print(merge_operno, merge_prodg1, merge_product)

None None None


In [125]:
class DataPreprocessorForInline:
    def __init__(self,
                 df: pyspark.sql.dataframe,
                 columns_list: list[str],
                 certain_column: str,
                 key_words: list[str],
                 convert_to_numeric_list: list[str],
                 merge_operno_list: List[Dict[str, List[str]]],
                 merge_prodg1_list: List[Dict[str, List[str]]],
                 merge_product_list: List[Dict[str, List[str]]]
                 ):
        self.df = df
        self.columns_list = columns_list
        self.certain_column = certain_column
        self.key_words = key_words
        self.convert_to_numeric_list = convert_to_numeric_list
        self.merge_operno_list = merge_operno_list
        self.merge_prodg1_list = merge_prodg1_list
        self.merge_product_list = merge_product_list

    @staticmethod
    def select_columns(df: pyspark.sql.dataframe, columns_list: list[str]) -> pyspark.sql.dataframe:
        return df.select(columns_list)

    @staticmethod
    def exclude_some_data(df: pyspark.sql.dataframe, key_words: list[str],
                          certain_column: str) -> pyspark.sql.dataframe:
        key_words_str = '|'.join(key_words)
        df_filtered = df.filter(~col(certain_column).rlike(key_words_str))
        return df_filtered

    @staticmethod
    def pre_process(df: pyspark.sql.dataframe, convert_to_numeric_list: list[str]) -> pyspark.sql.dataframe:
        for column in convert_to_numeric_list:
            df = df.withColumn(column, col(column).cast('double'))
        if 'SITE_COUNT' in convert_to_numeric_list:
            convert_to_numeric_list.remove('SITE_COUNT')
        df = df.dropna(subset=convert_to_numeric_list, how='all')
        return df

    @staticmethod
    def integrate_columns(df: pyspark.sql.dataframe,
                          merge_operno_list: List[Dict[str, List[str]]],
                          merge_prodg1_list: List[Dict[str, List[str]]],
                          merge_product_list: List[Dict[str, List[str]]]) -> pyspark.sql.dataframe:
        """
        Integrate columns in the DataFrame based on the provided list.

        :param df: The input DataFrame.
        :param merge_operno_list: A list of dictionaries where each dictionary contains values to be merged.
               Example: [{'2F.CDS10_XX.TDS01': ['2F.CDS10', 'XX.TDS01']},
                         {'2F.CDS20_XX.CDS20': ['2F.CDS20', 'XX.CDS20']}]
        :param merge_prodg1_list: A list of dictionaries for merging 'PRODG1' column in a similar fashion.
        :param merge_product_list: A list of dictionaries for merging 'PRODUCT_ID' column in a similar fashion.

        :return: DataFrame with 'OPER_NO' and other specified columns integrated according to the merge rules.
        """
        df_merged = DataPreprocessorForInline.integrate_single_column(df, merge_operno_list, 'OPER_NO')
        df_merged = DataPreprocessorForInline.integrate_single_column(df_merged, merge_prodg1_list, 'PRODG1')
        df_merged = DataPreprocessorForInline.integrate_single_column(df_merged, merge_product_list, 'PRODUCT_ID')
        return df_merged

    @staticmethod
    def integrate_single_column(df: pyspark.sql.dataframe,
                                merge_list: List[Dict[str, List[str]]],
                                column_name: str) -> pyspark.sql.dataframe:
        """
        Integrate columns in the DataFrame based on the provided list.

        :param df: The input DataFrame.
        :param merge_list: A list of dictionaries where each dictionary contains values to be merged.
        :param column_name: The name of the column to be merged.

        :return: DataFrame with specified column integrated according to the merge rules.
        """
        splitter_comma = ","
        if merge_list is not None and len(merge_list) > 0:
            values_to_replace = [list(rule.values())[0] for rule in merge_list]
            merged_values = [splitter_comma.join(list(rule.values())[0]) for rule in merge_list]

            for values, replacement_value in zip(values_to_replace, merged_values):
                df = df.withColumn(column_name,
                                   when(col(column_name).isin(values), replacement_value).otherwise(col(column_name)))
        return df

    def run(self) -> pyspark.sql.dataframe:
        df_select = self.select_columns(df=self.df, columns_list=self.columns_list)
        df_esd = self.exclude_some_data(df=df_select, key_words=self.key_words, certain_column=self.certain_column)
        
        df_integrate = self.integrate_columns(df=df_esd, 
                                              merge_operno_list=self.merge_operno_list,
                                              merge_prodg1_list=self.merge_prodg1_list,
                                              merge_product_list=self.merge_product_list)
        df_preprocess = self.pre_process(df=df_integrate, convert_to_numeric_list=self.convert_to_numeric_list)
        return df_integrate, df_preprocess

In [126]:
grpby_list = ['OPER_NO']

columns_list = grpby_list + ['WAFER_ID', 'INLINE_PARAMETER_ID', 'AVERAGE', 'MAX_VAL', 'MEDIAN', 'MIN_VAL',
                                 'STD_DEV', 'PERCENTILE_25', 'PERCENTILE_75', 'SITE_COUNT', 'label']

key_words = ['CXS', 'CYS', 'FDS']

convert_to_numeric_list = ['AVERAGE', 'MAX_VAL', 'MEDIAN', 'MIN_VAL', 'STD_DEV', 'PERCENTILE_25',
                               'PERCENTILE_75', 'SITE_COUNT']

certain_column = 'INLINE_PARAMETER_ID'

df_spark = df_spark.withColumnRenamed('OPE_NO', 'OPER_NO')
df_integrate, df_preprocess = DataPreprocessorForInline(df=df_spark,
                                          columns_list=columns_list,
                                          certain_column=certain_column,
                                          key_words=key_words,
                                          convert_to_numeric_list=convert_to_numeric_list,
                                          merge_operno_list=merge_operno,
                                          merge_prodg1_list=merge_prodg1,
                                          merge_product_list=merge_product).run()
print(f"df_integrate shape: ({df_integrate.count()}, {len(df_integrate.columns)})")
df_integrate.show()

print(f"df_preprocess shape: ({df_preprocess.count()}, {len(df_preprocess.columns)})")
df_preprocess.show()

df_integrate shape: (2451, 12)
+--------+---------+-------------------+----------------+--------+--------+--------+--------------+-------------+-------------+----------+-----+
| OPER_NO| WAFER_ID|INLINE_PARAMETER_ID|         AVERAGE| MAX_VAL|  MEDIAN| MIN_VAL|       STD_DEV|PERCENTILE_25|PERCENTILE_75|SITE_COUNT|label|
+--------+---------+-------------------+----------------+--------+--------+--------+--------------+-------------+-------------+----------+-----+
|1F.FQE10|NAZ439-03|               TAW1|        109.2519|    null|    null|    null|          null|         null|         null|      null|    0|
|1F.FQE10|NAZ439-03|               TAWB|        109.2519|    null|    null|    null|          null|         null|         null|      null|    0|
|1F.FQE10|NAZ439-03|               TDS0|        108.4117|108.4117|108.4117|108.4117|           0.0|     108.4117|     108.4117|       1.0|    0|
|1F.FQE10|NAZ439-03|               TDS1|109.251923529412|110.1915|109.1703|108.4117|0.547565949829|

In [122]:
result_orig = df_spark.groupBy('OPER_NO').agg(countDistinct('INLINE_PARAMETER_ID').alias('nunique'))
result_orig.show()

result_integrate = df_integrate.groupBy('OPER_NO').agg(countDistinct('INLINE_PARAMETER_ID').alias('nunique'))
result_integrate.show()

result_process = df_preprocess.groupBy('OPER_NO').agg(countDistinct('INLINE_PARAMETER_ID').alias('nunique'))
result_process.show()

+--------+-------+
| OPER_NO|nunique|
+--------+-------+
|1U.PQW10|     74|
|1V.EQW10|    117|
|1C.CDG10|     11|
|1U.EQW10|     97|
|1V.PQW10|    270|
|1F.FQE10|     15|
+--------+-------+

+--------+-------+
| OPER_NO|nunique|
+--------+-------+
|1U.PQW10|     74|
|1V.EQW10|    117|
|1C.CDG10|     11|
|1U.EQW10|     95|
|1V.PQW10|    270|
|1F.FQE10|     12|
+--------+-------+

+--------+-------+
| OPER_NO|nunique|
+--------+-------+
|1U.PQW10|     74|
|1V.EQW10|    116|
|1C.CDG10|     10|
|1U.EQW10|     94|
|1V.PQW10|    270|
|1F.FQE10|     12|
+--------+-------+



In [19]:
# dfg = df_pandas[df_pandas['OPE_NO'] == '1F.FQE10']
# dfg[dfg['INLINE_PARAMETER_ID'].str.contains('|'.join(['CXS', 'CYS', 'FDS']))]

In [127]:
class GetTrainDataForInline:
    def __init__(self, df: pyspark.sql.dataframe, grpby_list: list[str]):
        """
        Initialize the GetTrainDataForInline class.

        Parameters:
        - df (pyspark.sql.dataframe): Input DataFrame.
        - grpby_list (list): List of grouping columns.

        This class is designed to perform commonality analysis and retrieve training data based on the
        condition "bad_num >= 1 AND good_num >= 1" in each grpby_list, i.e. each OPE_NO for inline data.
        """
        self.df_run = df
        self.grpby_list = grpby_list

    @staticmethod
    def commonality_analysis(df_run: pyspark.sql.dataframe, grpby_list: list[str]) -> pyspark.sql.dataframe:
        grps_all = (df_run.groupBy(grpby_list)
                    .agg(countDistinct('WAFER_ID', when(df_run['label'] == 0, 1)).alias('good_num'),
                         countDistinct('WAFER_ID', when(df_run['label'] == 1, 1)).alias('bad_num'))
                    .na.fill(0))
        grps_all = grps_all.withColumn("conditions_satisfied",
                                       when((col('good_num') >= 1) & (col('bad_num') >= 1), True).otherwise(False))
        grps = grps_all.filter("good_num >= 1 AND bad_num >= 1")
        return grps_all, grps

    @staticmethod
    def get_data_list(common_res: pyspark.sql.dataframe, grpby_list: list[str]) -> List[Dict[str, str]]:
        data_list = common_res.select(grpby_list).collect()
        data_dict_list = [row.asDict() for row in data_list]
        return data_dict_list

    @staticmethod
    def get_train_data(df_run: pyspark.sql.dataframe, data_dict_list: List[Dict[str, str]]) -> pyspark.sql.dataframe:
        first_data_dict = data_dict_list[0]
        conditions = " AND ".join(["{} == '{}'".format(col_, first_data_dict[col_]) for col_ in first_data_dict])
        df_s = df_run.filter(conditions)
        for i in range(1, len(data_dict_list)):
            data_dict = data_dict_list[i]
            conditions = " AND ".join(["{} == '{}'".format(col_, data_dict[col_]) for col_ in data_dict])
            df_m = df_run.filter(conditions)
            df_s = df_s.union(df_m)
        return df_s
    
    def run(self) -> pyspark.sql.dataframe:
        grps_all, grps = self.commonality_analysis(df_run=self.df_run, grpby_list=self.grpby_list)
#         conditions_not_satisfied = grps_all.filter("conditions_satisfied==False").select(self.grpby_list + ["good_num", 'bad_num'])
#         parametric_for_unsatisfied_conditions = conditions_not_satisfied.join(self.df_run.select(self.grpby_list + ['INLINE_PARAMETER_ID']), self.grpby_list, "left").dropDuplicates()
        
        if grps.isEmpty():
            msg = f"按照{'+'.join(self.grpby_list)}分组后的数据, 没有组合满足条件good >= 1且bad >= 1, 无法进行分析."
            raise RCABaseException(msg)

        if grps_all.count() == grps.count():
            return grps_all, self.df_run
        else:
            data_dict_list = self.get_data_list(common_res=grps, grpby_list=self.grpby_list)
            train_data = self.get_train_data(df_run=self.df_run, data_dict_list=data_dict_list)
            return grps_all, train_data

In [128]:
grps_all, df_train = GetTrainDataForInline(df=df_preprocess, grpby_list=grpby_list).run()
print("grps_all:")
grps_all.show()

print(f"df_train shape: ({df_train.count()}, {len(df_train.columns)})")
df_train.show()

grps_all:
+--------+--------+-------+--------------------+
| OPER_NO|good_num|bad_num|conditions_satisfied|
+--------+--------+-------+--------------------+
|1U.PQW10|       2|      3|                true|
|1C.CDG10|       2|     13|                true|
|1V.EQW10|       2|      6|                true|
|1U.EQW10|       2|      4|                true|
|1V.PQW10|       2|      5|                true|
|1F.FQE10|       1|      1|                true|
+--------+--------+-------+--------------------+

df_train shape: (2433, 12)
+--------+---------+-------------------+----------------+--------+--------+--------+--------------+-------------+-------------+----------+-----+
| OPER_NO| WAFER_ID|INLINE_PARAMETER_ID|         AVERAGE| MAX_VAL|  MEDIAN| MIN_VAL|       STD_DEV|PERCENTILE_25|PERCENTILE_75|SITE_COUNT|label|
+--------+---------+-------------------+----------------+--------+--------+--------+--------------+-------------+-------------+----------+-----+
|1F.FQE10|NAZ439-03|               TA

In [44]:
# df_preprocess_pandas = df_preprocess.toPandas()

In [45]:
# oper = '1C.CDG10'
# df_run2 = df_preprocess_pandas.query(f"OPER_NO == '{oper}'") 
# df_run2['INLINE_PARAMETER_ID'].value_counts()

In [46]:
# grps_all, grps = GetTrainDataForInline.commonality_analysis(df_run=df_preprocess, grpby_list=grpby_list)

# grps_all.show()
# grps.show()

In [47]:
# conditions_not_satisfied = grps_all.filter("conditions_satisfied==False").select(grpby_list + ["good_num", 'bad_num'])
# print("conditions_not_satisfied:")
# conditions_not_satisfied.show()

In [48]:
# parametric_for_unsatisfied_conditions = conditions_not_satisfied.join(df_preprocess.select(grpby_list + ['INLINE_PARAMETER_ID']), grpby_list, "left").dropDuplicates()
# print(parametric_for_unsatisfied_conditions.count())
# parametric_for_unsatisfied_conditions.show()

In [49]:
# data_dict_list = GetTrainDataForInline.get_data_list(common_res=grps, grpby_list=grpby_list)
# print(data_dict_list)

# train_data = GetTrainDataForInline.get_train_data(df_run=df_preprocess, data_dict_list=data_dict_list)
# print(train_data.count())
# train_data.show()

---- 

In [129]:
class FitInlineModelByWafer:
    def __init__(self,
                 df: pyspark.sql.dataframe,
                 grpby_list: list[str],
                 columns_to_process: list[str],
                 missing_value_threshold: float,
                 model: str = 'pca'):
        self.df = df
        self.grpby_list = grpby_list
        self.columns_to_process = columns_to_process
        self.missing_value_threshold = missing_value_threshold
        self.model = model

    @staticmethod
    def process_missing_values(df, columns_to_process, missing_value_threshold):
        for column in columns_to_process:
            missing_percentage = df[column].isnull().mean()
            if missing_percentage > missing_value_threshold:
                df = df.drop(columns=[column])
            else:
                df[column] = df[column].fillna(df[column].mean())
        return df

    @staticmethod
    def get_pivot_table(df, grpby_list, columns_to_process, missing_value_threshold):
        df_specific = FitInlineModelByWafer.process_missing_values(df, columns_to_process, missing_value_threshold)
        index_list = ['WAFER_ID', 'label']
        columns_list = grpby_list + ['INLINE_PARAMETER_ID']
        values_list = df_specific.columns.difference(
            ['WAFER_ID', 'INLINE_PARAMETER_ID', 'SITE_COUNT', 'label'] + grpby_list)
        pivot_result = df_specific.pivot_table(index=index_list,
                                               columns=columns_list,
                                               values=values_list)
        pivot_result.columns = pivot_result.columns.map('#'.join)
        pivot_result = FitInlineModelByWafer.process_missing_values(pivot_result, pivot_result.columns,
                                                                    missing_value_threshold)
        pivot_result = pivot_result.reset_index(drop=False)
        # Remove completely identical columns
        for column in pivot_result.columns.difference(index_list):
            if pivot_result[column].nunique() == 1:
                pivot_result = pivot_result.drop(column, axis=1)
        return pivot_result

    @staticmethod
    def fit_pca_model(df, grpby_list, columns_to_process, missing_value_threshold):
        schema_all = StructType([StructField("features", StringType(), True),
                                 StructField("importance", FloatType(), True),
                                 StructField("algorithm_satisfied", StringType(), True),
                                 StructField("x_train_shape", StringType(), True)])

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_model_result(df_run):
            pivot_result = FitInlineModelByWafer.get_pivot_table(df=df_run,
                                                                 grpby_list=grpby_list,
                                                                 columns_to_process=columns_to_process,
                                                                 missing_value_threshold=missing_value_threshold)

            x_train = pivot_result[pivot_result.columns.difference(['WAFER_ID', 'label']).tolist()]

            if min(x_train.shape) > 2:
                res_top_select = FitInlineModelByWafer.construct_features_when_satisfy_pca(x_train=x_train)
                return res_top_select
            else:
                res_top_select = FitInlineModelByWafer.construct_features_when_not_satisfied(x_train=x_train)
                return res_top_select

        return df.groupby(grpby_list).apply(get_model_result)

    @staticmethod
    def construct_features_when_not_satisfied(x_train):
        x_len = len(x_train.columns)
        res_top_select = pd.DataFrame({"features": x_train.columns,
                                       "importance": [0.0] * x_len,
                                       "algorithm_satisfied": ['FALSE'] * x_len,
                                       "x_train_shape": [str(x_train.shape)] * x_len})
        return res_top_select

    @staticmethod
    def construct_features_when_satisfy_pca(x_train):
        # 得到PCA算法结果res_top_select
        n_components = min(min(x_train.shape) - 2, 20)
        model = pca(n_components=n_components, verbose=None)
        results = model.fit_transform(x_train)
        res_top = results['topfeat']
        res_top_select = res_top[res_top['type'] == 'best'][['feature', 'loading']]
        res_top_select['importance'] = abs(res_top_select['loading'])
        res_top_select = res_top_select.rename(columns={'feature': 'features'}).drop("loading",
                                                                                     axis=1).drop_duplicates()
        # 增加算法为0的features
        has_importance_features = res_top_select['features'].values
        zero_importance_features = x_train.columns.difference(has_importance_features).to_list()
        len_f = len(zero_importance_features)
        zero_df  = pd.DataFrame({'features': zero_importance_features, 'importance': [0.0] * len_f})
        res_top_select = res_top_select.append(zero_df, ignore_index=True)
        
        # 合并二者的结果
        res_top_select['x_train_shape'] = str(x_train.shape)
        res_top_select['algorithm_satisfied'] = 'TRUE'
        return res_top_select

    def run(self):
        if self.model == 'pca':
            res = self.fit_pca_model(df=self.df, grpby_list=self.grpby_list,
                                     columns_to_process=self.columns_to_process,
                                     missing_value_threshold=self.missing_value_threshold)

        else:
            res = self.fit_classification_model(df=self.df, grpby_list=self.grpby_list,
                                                columns_to_process=self.columns_to_process,
                                                missing_value_threshold=self.missing_value_threshold,
                                                model=self.model)
        return res

In [130]:
res = FitInlineModelByWafer(df=df_train,
                            grpby_list=grpby_list,
                            columns_to_process=['AVERAGE', 'MAX_VAL', 'MEDIAN', 'MIN_VAL', 'STD_DEV',
                                                'PERCENTILE_25', 'PERCENTILE_75'],
                            missing_value_threshold=0.6,
                            model='pca').run()

m = res.filter('algorithm_satisfied==TRUE').count()
print(m)

print(f"res shape: ({res.count()}, {len(res.columns)})")
res.show(50)
print(res.dtypes)

D:\developer\Anaconda\lib\site-packages\pyspark\sql\pandas\group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


411
res shape: (418, 4)
+--------------------+----------+-------------------+-------------+
|            features|importance|algorithm_satisfied|x_train_shape|
+--------------------+----------+-------------------+-------------+
|AVERAGE#1C.CDG10#...| 0.9999995|               TRUE|     (15, 10)|
|AVERAGE#1C.CDG10#...|  0.782774|               TRUE|     (15, 10)|
|AVERAGE#1C.CDG10#...|0.77940005|               TRUE|     (15, 10)|
|AVERAGE#1C.CDG10#...|0.94151205|               TRUE|     (15, 10)|
|AVERAGE#1C.CDG10#...| 0.7007957|               TRUE|     (15, 10)|
|AVERAGE#1C.CDG10#...|0.70067024|               TRUE|     (15, 10)|
|AVERAGE#1C.CDG10#...|0.96751493|               TRUE|     (15, 10)|
|AVERAGE#1C.CDG10#...| 0.9823262|               TRUE|     (15, 10)|
|AVERAGE#1C.CDG10#...|       0.0|               TRUE|     (15, 10)|
|AVERAGE#1C.CDG10#...|       0.0|               TRUE|     (15, 10)|
|AVERAGE#1C.CDG10#...|       0.0|               TRUE|     (15, 10)|
|AVERAGE#1F.FQE10#...|  

In [61]:
# oper = '1C.CDG10'
# df_run = df_train_pandas.query(f"OPER_NO == '{oper}'") 
# df_run

In [62]:
# df_run['INLINE_PARAMETER_ID'].value_counts()

In [63]:
# index_list = ['WAFER_ID', 'label']
# columns_list = grpby_list + ['INLINE_PARAMETER_ID']
# values_list = df_run.columns.difference(
#     ['WAFER_ID', 'INLINE_PARAMETER_ID', 'SITE_COUNT', 'label'] + grpby_list)
# pivot_result = df_run.pivot_table(index=index_list,
#                                        columns=columns_list,
#                                        values=values_list)
# pivot_result

In [64]:
# pivot_result = FitInlineModelByWafer.get_pivot_table(df=df_run,
#                                                      grpby_list=grpby_list,
#                                                      columns_to_process=['AVERAGE', 'MAX_VAL', 'MEDIAN', 'MIN_VAL', 'STD_DEV',
#                                                                 'PERCENTILE_25', 'PERCENTILE_75'],
#                                                      missing_value_threshold=0.6)

# x_train = pivot_result[pivot_result.columns.difference(['WAFER_ID', 'label']).tolist()]
# x_train

In [97]:
# x_train = pd.DataFrame()
# FitInlineModelByWafer.construct_features_when_not_satisfied(x_train=x_train)

In [50]:
# n_components = min(min(x_train.shape) - 2, 20)
# model = pca(n_components=n_components, verbose=None)
# results = model.fit_transform(x_train)
# res_top = results['topfeat']
# res_top

In [65]:
# res_top_select = FitInlineModelByWafer.construct_features_when_satisfy_pca(x_train=x_train)
# res_top_select

In [98]:
# has_importance_features = res_top_select['features'].values
# has_importance_features

In [99]:
# zero_importance_features = x_train.columns.difference(has_importance_features).to_list()
# len_f = len(zero_importance_features)

# zero_df  = pd.DataFrame({'features': zero_importance_features,
#                             'importance': [0.0] * len_f})
# zero_df

In [100]:
# res_top_select.append(zero_df, ignore_index=True)

In [56]:
# res = res.withColumn("algorithm_satisfied", res["algorithm_satisfied"].cast(StringType()))
# res_pandas = res.toPandas()
# res_pandas.dtypes

In [57]:
# res_pandas.to_csv("D:/Jupyterfiles/晶合MVAFDC_general开发/MVAanlysisDevelop/inline_algorithm/inline调试1/bywafer调试2-1/res.csv")

----

In [103]:
class SplitInlineModelResults:
    def __init__(self, df: pyspark.sql.dataframe, grpby_list: List[str], request_id: str,
                 grps_all: pyspark.sql.dataframe, unique_params_within_groups: pyspark.sql.dataframe):
        self.df = df
        self.grpby_list = grpby_list
        self.request_id = request_id
        self.grps_all = grps_all
        self.unique_params_within_groups= unique_params_within_groups

    @staticmethod
    def split_features(df: pd.DataFrame, index: int) -> str:
        return df['features'].apply(lambda x: x.split('#')[index])

    @staticmethod
    def get_split_features(df: pd.DataFrame, grpby_list: List[str]) -> pd.DataFrame:
        n_feats = len(grpby_list)
        for i in range(n_feats):
            df[grpby_list[i]] = SplitInlineModelResults.split_features(df, i + 1)

        df['INLINE_PARAMETER_ID'] = SplitInlineModelResults.split_features(df, n_feats + 1)
        df = df.drop(['features'], axis=1).reset_index(drop=True)
        return df

    @staticmethod
    def split_calculate_features(df: pyspark.sql.dataframe, grpby_list: List[str], by: str) -> pyspark.sql.dataframe:
        struct_fields = [StructField(col_, StringType(), True) for col_ in grpby_list]
        struct_fields.extend([StructField("INLINE_PARAMETER_ID", StringType(), True),
                              StructField("importance", FloatType(), True),
                              StructField("algorithm_satisfied", StringType(), True)])
        schema_all = StructType(struct_fields)

        @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
        def get_model_result(df_run):
            split_table = SplitInlineModelResults.get_split_features(df=df_run, grpby_list=grpby_list)
            split_table_grpby = split_table.groupby(grpby_list + ['INLINE_PARAMETER_ID', 'algorithm_satisfied'])[
                'importance'].sum().reset_index(drop=False)
            return split_table_grpby

        return df.groupby(by).apply(get_model_result)

    @staticmethod
    def add_certain_column(df: pyspark.sql.dataframe, grpby_list, request_id: str):
        total_importance = df.select(spark_sum("importance")).collect()[0][0]
        df = (df.withColumn("weight", col("importance") / total_importance)
                .withColumn("weight_percent", col("weight") * 100))

        df = df.drop('importance')
        df = df.orderBy(col("weight").desc())
        df = (df.withColumn("good_num", df["good_num"].cast(FloatType()))
              .withColumn("bad_num", df["bad_num"].cast(FloatType()))
              .withColumn('index_no', monotonically_increasing_id() + 1)
              .withColumn("request_id", lit(request_id))
              .withColumn("AVG_SPEC_CHK_RESULT_COUNT", lit(0.0)))

        info_list = ['PRODUCT_ID', 'OPER_NO', 'PRODG1']
        for column in info_list:
            if column not in df.columns:
                df = df.withColumn(column, lit(None).cast(StringType()))
        return df

    def run(self):
        df = self.df.withColumn('temp', lit(0))
        split_res = self.split_calculate_features(df=df, grpby_list=self.grpby_list, by='temp')
        print("split_res:", split_res.count())
        split_res.show()
        
        split_res = split_res.drop('algorithm_satisfied')
        missing_rows = self.unique_params_within_groups.join(split_res, 
                                                                on=self.grpby_list + ['INLINE_PARAMETER_ID'], 
                                                                how='left_anti')
        missing_rows = missing_rows.withColumn('importance', lit(0))
        print("missing_rows:")
        missing_rows.show()
        
        split_res_update_missing_features = split_res.unionByName(missing_rows, allowMissingColumns=True)
        print("split_res_update_missing_features:", split_res_update_missing_features.count())
        split_res_update_missing_features.show()
        
        split_res_update_wafer_num = split_res_update_missing_features.join(self.grps_all.select(self.grpby_list + ["good_num", 'bad_num']), 
                                                                            on=self.grpby_list, how='left')
        print("split_res_update_wafer_num:", split_res_update_wafer_num.count()) 
        split_res_update_wafer_num.show()
        
        
#         final_res_pandas = split_res_update_wafer_num.toPandas()
#         final_res_pandas.to_csv("D:/Jupyterfiles/晶合MVAFDC_general开发/MVAanlysisDevelop/inline_algorithm/inline调试1/bywafer调试2-1/split_res_update1.csv")
        
#         split_res_update = split_res.unionByName(self.unique_params_within_groups, allowMissingColumns=True)
#         split_res_update = split_res_update.dropDuplicates(self.grpby_list + ['INLINE_PARAMETER_ID'])
#         print("split_res_update:", split_res_update.count())
#         split_res_update.show()
        
        final_res = self.add_certain_column(df=split_res_update_wafer_num, 
                                            grpby_list = self.grpby_list,
                                            request_id=self.request_id)
        print("final_res:", final_res.count())
        final_res.show()
        return final_res

In [118]:
grpby_list = ['OPER_NO', 'PRODUCT_ID']
grouped = df_integrate.groupby(*grpby_list).agg(collect_set('INLINE_PARAMETER_ID').alias('unique_values'))
exploded = grouped.select(*grpby_list, explode(col('unique_values')).alias('INLINE_PARAMETER_ID'))
unique_params_within_groups = exploded.dropDuplicates()
print(unique_params_within_groups.count())
unique_params_within_groups.show()

813
+--------+--------------+-------------------+
| OPER_NO|    PRODUCT_ID|INLINE_PARAMETER_ID|
+--------+--------------+-------------------+
|1C.CDG10|AFPNM301N.0B01|               TSW0|
|1C.CDG10|AFPNM301N.0B01|               HFT0|
|1C.CDG10|AFPNM301N.0B01|               REW0|
|1C.CDG10|AFPNM301N.0B01|               OEW0|
|1C.CDG10|AFPNM301N.0B01|               SFW0|
|1C.CDG10|AFPNM301N.0B01|               SEW0|
|1C.CDG10|AFPNM301N.0B01|               PTW0|
|1C.CDG10|AFPNM301N.0B01|               CIW0|
|1C.CDG10|AFPNM301N.0B01|               EEW0|
|1C.CDG10|AFPNM301N.0B01|               PEW0|
|1C.CDG10|AFPNM301N.0B01|               FEW0|
|1F.FQE10|AFPNM301N.0A01|               TDS1|
|1F.FQE10|AFPNM301N.0A01|               TUW9|
|1F.FQE10|AFPNM301N.0A01|               THW1|
|1F.FQE10|AFPNM301N.0A01|               TAWB|
|1F.FQE10|AFPNM301N.0A01|               TUWB|
|1F.FQE10|AFPNM301N.0A01|               TUW1|
|1F.FQE10|AFPNM301N.0A01|               TAW1|
|1F.FQE10|AFPNM301N.0A01|     

In [104]:
final_res = SplitInlineModelResults(df=res, grpby_list=grpby_list, request_id='inline',
                                    grps_all=grps_all, unique_params_within_groups=unique_params_within_groups).run()

D:\developer\Anaconda\lib\site-packages\pyspark\sql\pandas\group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


split_res: 415
+--------+-------------------+----------+-------------------+
| OPER_NO|INLINE_PARAMETER_ID|importance|algorithm_satisfied|
+--------+-------------------+----------+-------------------+
|1C.CDG10|               EEW0|0.96751493|               TRUE|
|1C.CDG10|               FEW0| 1.4014659|               TRUE|
|1C.CDG10|               HFT0|0.94151205|               TRUE|
|1C.CDG10|               OEW0|       0.0|               TRUE|
|1C.CDG10|               PEW0|       0.0|               TRUE|
|1C.CDG10|               PTW0| 0.9999995|               TRUE|
|1C.CDG10|               REW0|  0.782774|               TRUE|
|1C.CDG10|               SEW0|       0.0|               TRUE|
|1C.CDG10|               SFW0|0.77940005|               TRUE|
|1C.CDG10|               TSW0| 0.9823262|               TRUE|
|1F.FQE10|               TAW1|       0.0|              FALSE|
|1F.FQE10|               TDS0|       0.0|              FALSE|
|1F.FQE10|               TDS1|       0.0|              

In [105]:
final_res_pandas = final_res.toPandas()
final_res_pandas

,OPER_NO,INLINE_PARAMETER_ID,good_num,bad_num,weight,weight_percent,index_no,request_id,AVG_SPEC_CHK_RESULT_COUNT,PRODUCT_ID,PRODG1
0,1C.CDG10,FEW0,2.0,13.0,0.108466,10.846564,1,inline,0.0,None,None
1,1V.EQW10,40W0,2.0,6.0,0.080927,8.092673,2,inline,0.0,None,None
2,1C.CDG10,PTW0,2.0,13.0,0.077394,7.739438,3,inline,0.0,None,None
3,1C.CDG10,TSW0,2.0,13.0,0.076027,7.602657,4,inline,0.0,None,None
4,1C.CDG10,EEW0,2.0,13.0,0.074880,7.488026,5,inline,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...
574,1V.PQW10,234W0,2.0,5.0,0.000000,0.000000,575,inline,0.0,None,None
575,1V.PQW10,224W0,2.0,5.0,0.000000,0.000000,576,inline,0.0,None,None
576,1V.PQW10,240W0,2.0,5.0,0.000000,0.000000,577,inline,0.0,None,None
577,1V.PQW10,233W0,2.0,5.0,0.000000,0.000000,578,inline,0.0,None,None


In [106]:
final_res_pandas.to_csv("D:/Jupyterfiles/晶合MVAFDC_general开发/MVAanlysisDevelop/inline_algorithm/inline调试1/bywafer调试2-1/final_res.csv")

In [107]:
74+117+11+95+270+12

579